# Survey - Global Wheat Detection

- 小麦の穂の位置検出コンペ
- https://www.kaggle.com/c/global-wheat-detection

<img src="https://user-images.githubusercontent.com/846237/112433696-f4133b80-8d85-11eb-9f36-b6d20fdeab3e.png">


### 開催期間

- 2020.05.05 〜 2020.08.19


In [1]:
3+5

8

## 今回の要点

- 物体検出には<font color="red">Yolo v5</font>がオススメ
- ドメインの軽減には<font color="red">Psedo Labeling</font>が有効
- 矩形情報のアンサンブルには<font color="red">WBF</font>

## どんなコンペなの？

小麦の画像が与えられるので，穂の箇所を矩形で囲む


### 入力例

<img src="https://user-images.githubusercontent.com/846237/113079521-2bfc0200-9210-11eb-8e68-2e3b955f3c94.png" width="600">


### 出力例

<img src="https://user-images.githubusercontent.com/846237/112435514-2de54180-8d88-11eb-8d2d-7714e1519900.png" width="600">

### どんなデータがあるの？

- 📄 `train.csv` - image_id, width(画像の幅), height(画像の高さ), bbox(矩形情報)
    - 画像サイズはすべて1024x1024
    - `bbox`は`[xmin, ymin, width, height]`
- 📄 `sample_submission.csv` - 
- 📂 `train/` - 訓練用の画像フォルダ, <font color="green">3300以上</font>の画像がある. 
- 📂 `test/` - テスト用の画像フォルダ, コード提出形式でダウンロードできるのは10枚のみ

## 外部データ

- SPIKE dataset: https://www.kaggle.com/c/global-wheat-detection/discussion/164346
- https://plantimages.nottingham.ac.uk/

## どうやって評価するの？

- [IoU](https://www.kaggle.com/c/global-wheat-detection/overview/evaluation)

<img src="https://user-images.githubusercontent.com/846237/113235060-67b0cd80-92dd-11eb-8f36-2e10d9188b1e.png" width="600">

## 検出できると何が嬉しいの？

- 農家の収穫時の品質改善や研究者への穂の密度や大きさの情報を提供できるようになる
    - つまり，シリアルやトーストなど食事が美味しくなる
    - cf. https://www.kaggle.com/c/global-wheat-detection/overview/description

<img src="https://user-images.githubusercontent.com/846237/113287047-fb0ef080-9327-11eb-9afc-4c97e1adbe89.png" width="600">

## このコンペの難しいところ

- 世界中の小麦の穂の画像を集めているので，品種や地域，育成度合いや形がバラバラ
- 学習データと評価用のデータに共通となる品種&地域はない
    - <font color="red">未知の環境でも運用</font>できるモデルを構築する必要がある
- アノテーターのラベルミスがかなりある

<img src="https://user-images.githubusercontent.com/846237/112433696-f4133b80-8d85-11eb-9f36-b6d20fdeab3e.png" width="600">

## いかにして汎化性能を上げるか？

- テストデータは品種も地域も違うので，いかにモデルに品種の違いの情報を組み込むかがポイントとなる
- 上位が当然のようにやってるのがPseudo Labeling(疑似ラベリング)

### やり方

1. 教師となるモデルを訓練データで学習させる
2. 教師モデルで提出時にテストデータへ疑似ラベルをつける
3. 教師となるモデルより大きいモデルを用意する
4. 生徒モデルに訓練データ+疑似ラベル付きテストデータで学習させる
5. 学習させたモデルで提出する

<img src="https://user-images.githubusercontent.com/846237/113270645-ad3cbd00-9314-11eb-96d0-8888b59824e2.png" width="400">
<img src="https://user-images.githubusercontent.com/846237/113270572-97c79300-9314-11eb-9298-340f80ff9434.png" width="400">
<img src="https://user-images.githubusercontent.com/846237/113270666-b3329e00-9314-11eb-9196-c710d3e1ad3b.png" width="400">

- さきほどのやり方を洗練させた(ノイズを加えた)方法が`Noisy Student`
    - この記事がわかりやすい.
        - cf. https://ai-scholar.tech/articles/treatise/noisy-student-ai-379
        - cf. https://qiita.com/rabbitcaptain/items/a15591ca49dc428223ca
    - 追加でやることとしては，ノイズを加えるだけ
    - DataAugmentaionの手法を選択する方法[RandAugment](https://arxiv.org/abs/1909.13719)を利用
    - 直近開かれたコンペだと当たり前のように使われている
<img src="https://user-images.githubusercontent.com/846237/113269422-63070c00-9313-11eb-8c44-8c030b49a447.png" width="600">

<img src="https://user-images.githubusercontent.com/846237/113270850-df4e1f00-9314-11eb-968c-e08096c5150c.png" width="400">

## ❏ 予備知識

- 物体検出のおおまかな流れ
- Models
    - [Yolo v5](https://github.com/ultralytics/yolov5)
        - TODO(nishimori-m): スターターキットのリンクを追加する
    - EfficientDet
    - ResNeSt
        - 物体検出でのbackbone
            - cf. https://qiita.com/takoroy/items/07c5039ab12b74137626
- Post Processing
    - WBF(Weighted boxes fusion)
        - https://github.com/ZFTurbo/Weighted-Boxes-Fusion
        - 物体検出での矩形情報のアンサンブル方法
    - PL(Pseudo Labeling)
        - https://www.kaggle.com/ufownl/global-wheat-detection-pseudo-labaling
- EMA
- Feature Pyramid Net(FPN)
    - cf. https://qiita.com/TaigaHasegawa/items/653abc81ac4ee1f0d7b8
- Data Augmentations
    - clahe
        - ヒストグラムが均等になるようコントラストを調整
            - 暗いところの小麦の穂を見えやすくするため?
    - mixup
        - cf. [新たなdata augmentation手法mixupを試してみた](https://qiita.com/yu4u/items/70aa007346ec73b7ff05)
    - CoarseDropout
        - cf. https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/169721
     - GridMask
         - cf. https://ohke.hateblo.jp/entry/2020/06/27/230000
 

## 物体検出ってどうやって学べばいいの？

- Deep Learningが叫ばれる前から研究されていた分野なので，歴史を知りつつ学んだほうが良い気がします．

物体検出の歴史については下記リンクがわかりやすいです

- [物体検出についての歴史まとめ(1)](https://qiita.com/mshinoda88/items/9770ee671ea27f2c81a9)
- [物体検出についての歴史まとめ(2)](https://qiita.com/mshinoda88/items/c7e0967923e3ed47fee5)

## よく利用されている物体検出アルゴリズム

- ライセンス的に許されるならYolo v5を使えば間違いなさそう(2021年3月時点)
- 次善がEfficientDet

- アルゴリズム
    - 🏆 Yolo v5(3rd)
    - 🥈 EfficientDet(1st, 2nd, 6th)
    - Faster RCNN
- backborn
    - ResNeSt
    
<img src="https://user-images.githubusercontent.com/26833433/103594689-455e0e00-4eae-11eb-9cdf-7d753e2ceeeb.png" width="600">

## 物体検出のツールキット

- [Detectron2](https://github.com/facebookresearch/detectron2)
    - Facebook AI Researchによる物体検出ライブラリ
- [MMDetection](https://github.com/open-mmlab/mmdetection)
    - 物体検出用のツールボックス
        - Faster RCNNやMask RCNNなど様々なベンチマーク用のアルゴリズムが使えるようになっている

## Boxes fusion

- 🏆  WBF(Weighted boxes fusion)
    - https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset
    - 矩形データのアンサンブル時に利用
- NMS
- Soft NMS

<img src="https://user-images.githubusercontent.com/846237/113249792-4c07f000-92fa-11eb-86ec-e8758455b4ac.png">

## 学習時の画像サイズ

- 1024x1024(1st, 2nd, 5th)
- 768x768(1st)
- 512x512(3rd)

3rdは性能の良いYolo v5を使っちゃってるので，サイズとしてはそのままの1024x1024が良さそう

## Data Augmentation

- CutMix(後述)
- Custom mosaic augmentation(後述)
- MixUp
- crop, hue, random brightness/contrast
- to gray
- vertical and horizontal flip
- random rotate 90
- cutout

### CutMix

- Data augmentationの一種
- Cutout + Mixupを行って一枚の画像にする

<img src="https://user-images.githubusercontent.com/846237/113252174-73f95280-92fe-11eb-861c-25c6210acf61.png">

cf. https://github.com/clovaai/CutMix-PyTorch

### Custom mosaic augmentation

- cf. https://www.kaggle.com/c/global-wheat-detection/discussion/172418
- 4つの画像を切り取って一枚の画像を作成
- 1stが試した方法
- CutMixの代わりに利用

<img src="https://user-images.githubusercontent.com/846237/112433819-23c24380-8d86-11eb-8ab3-a86370377bf2.png" width="600">


## Optimizer

- AdamW
- 余談: 最近，SAMと呼ばれるオプティマイザが良いらしい．
    - cf. [SoTAを総なめ！衝撃のオプティマイザー「SAM」爆誕&解説！](https://qiita.com/omiita/items/f24e4f06ae89115d248e)


## Learning Rate Scheduler

- [Cosine Annealing](https://paperswithcode.com/method/cosine-annealing)

<img src="https://user-images.githubusercontent.com/846237/113264379-ddcd2880-930d-11eb-8b1f-7339878b9a63.png" width="400">

## コンペ特有の知識

- ジグソーパズル
    - このコンペでは大きな画像を分割することでデータセットを作成している
        - つなぎ合わせることで，性能改善が可能
    - cf. https://github.com/lRomul/argus-tgs-salt/blob/master/mosaic/create_mosaic.py
    - 塩コンペでも同様のテクニックが有効だった
        - https://www.kaggle.com/c/tgs-salt-identification-challenge/overview

## このコンペで効果的なこと

- 上位陣のモデルはYolo v5かEfficientDetのほぼ2択
- 検出時のしきい値を低く設定(0.1)するとスコアが改善
- 1stと2ndはData Augmentationと外部データの差が大きそう


## 注意事項

- MITライセンスのみ可ということで，性能が良いがGNUライセンスであるYolo v5を使って良いか結構揉めた
    - TensorflowとかPyTorchも内部のライブラリにはMITライセンス以外が使われているけど，それは良いのかどうか

## スターターキット

- Yolo v5 + Pseudo Labeling
    - https://www.kaggle.com/nvnnghia/yolov5-pseudo-labeling/notebook

## Top Solutions


- 1st - https://www.kaggle.com/c/global-wheat-detection/discussion/172418
    - code
        - https://github.com/rwightman/efficientdet-pytorch
    - dataset
        - image solution
            - 1024x1024 for EfficientDet D7(Fold 0,1,2,3,4)
            - 768x768 for EfficientDet D7(Fold 1, 3)
            - 512x512 for EfficientDet D5(Fold 4)
        - external data
            - https://www.kaggle.com/c/global-wheat-detection/discussion/164346)
            - https://plantimages.nottingham.ac.uk/
        - cleaning
            - 縦横のどちらかが10px以下の矩形は除外
    - models
        - <font color='red'>EfficientDet D5&D7</font>, <font color='red'>Faster RCNN FPN</font>
    - train
        - batch size
            - 8: EfficientDet D7(image size: 768)
            - 20: Faster FPN ResNet 152(image size: 1024)
        - optimizer
            - adam for EfficientDet
                - learning rate: 5e-4
            - sgd for FasterRCNN
                - learning rate: 5e-3
        - LR scheduler:
            - cosine-annealing
        - augmentation
            - Custom mosaic augmentation
            - MixUp
    - ensemble
        - 8 TTA(vflip x hflip x rotate90)

- 2nd - https://www.kaggle.com/c/global-wheat-detection/discussion/175961
    - code
        - https://github.com/liaopeiyuan/TransferDet
    - model
        - <font color='red'>EfficientDet - D6</font>(pretrained w/ COCO dataset)
            - D4〜D7まで試してD6が性能が一番良かった
    - dataset
        - image resolution: 1024x1024
    - train
        - augmentation
            - crop, hue, random brightness/contrast
            - to gray
            - vertical and horizontal flip
            - random rotate 90
            - cutout

- 3rd - https://www.kaggle.com/c/global-wheat-detection/discussion/179055
    - code
        - https://github.com/ufownl/global-wheat-detection
    - model
        - <font color='red'>Yolo v5</font>
            - backbone: Darknet 53
    - train
        - max epochs: 9
        - learning rate: 0.001
        - batch size: 8
        - image resolution: 512x512
        - threshold: 0.1(物体検出のためのしきい値)
        - pseudo labeling
            - cf. https://www.kaggle.com/ufownl/global-wheat-detection-pseudo-labaling
    - post processing
        - TTAにWBF(Weighted boxes fusion)を利用

- 5th - https://www.kaggle.com/c/global-wheat-detection/discussion/172458
    - blog(日本語)
        - https://acro-engineer.hatenablog.com/entry/2020/08/21/175617
    - dataset
        - image resolution: 1024x1024
    - models
        - <font color='red'>EfficientDet D3 - D5</font>
    - train
        - 5 folds
        - optimizer: AdamW
        - max epoichs: 100
        - batch size: 640
        - pseudo labeling
        - augmentation:
            - mixup, mosaic, scale
            - hue, brightness
            - grid mask
                - cf. https://arxiv.org/abs/2001.04086
    - misc
        - threshold: 検出時のしきい値を0.1にすれば，0.695出ていたらしい

- 9th - https://www.kaggle.com/c/global-wheat-detection/discussion/172569
    - code: 
        - https://github.com/amirassov/kaggle-global-wheat-detection
    - dataset
        - external data
            - https://www.kaggle.com/c/global-wheat-detection/discussion/164346
   　- model
        - <font color='red'>MMDetection</font> 
            - cf. https://github.com/open-mmlab/mmdetection
    - train:
        - augmentaion
            - hflip, scale, shift, rotate90
            - brightness/contrast, hue saturation, rgb shift
            - random gamma
            - clahe
            - blur, motion blur
            - gauss noise
            - image compression
            - coarse dropout
            
<img src="https://user-images.githubusercontent.com/846237/113238334-8ade7b80-92e3-11eb-840a-027a86d4fcc7.png" width="600">
<img src="https://user-images.githubusercontent.com/846237/113238321-81edaa00-92e3-11eb-879e-be5e2cdc0d13.png">

- 16th - https://www.kaggle.com/c/global-wheat-detection/discussion/172567
    - train
        - augmentaiton
            - scale, mosaic, mixup, crop, cutout, hue, brightness, clahe

## Top Notebooks

- 2nd, https://www.kaggle.com/alexanderliao/effdet-d6-pl-s-bn-r-bb-a3-usa-eval-94-13-db
    - EfficientDet - D6
- 3rd, https://www.kaggle.com/x2x21x21x21/3rd-place-solution
- 6th - https://www.kaggle.com/stonewst98/what-a-pity-only-0-0001-away-from-0-77
    - Effficient Det
- 9th - https://www.kaggle.com/amiras/pseudo-ensemble-detectors-3-st-universenet-r10
- 13th - https://www.kaggle.com/dpyrtfq2372/efficientdet-with-double-psudo-labeling

## 次にやると良さそうなコンペ

- VinBigData Chest X-ray Abnormalities Detection
   - 物体検出コンペ
   - 医師ごとに矩形の領域がバラバラなので，どう統合していくか学べます
   - 胸部X線からの異常所見の分類
   - https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection